In [9]:
import pandas as pd

#pd.read_csv()  is for csv like excel like files
#pd.read_table() is for text files and then you can directly read the data in
df = pd.DataFrame(pd.read_table('ospC-NE.diff'))
df
#converter = pd.read_csv('ospC-NE.diff')

#converter

,seq_1,seq_2,len_gapless,len_variable,diff_gapless,diff_variable,len_aln,identitty,fac_diff,frac_gapless,frac_variable
0,A,B,209,115,40,43,217,80.8612,0.1914,0.1914,0.3739
1,A,C,209,115,38,40,217,81.8182,0.1818,0.1818,0.3478
2,A,D,210,115,47,48,217,77.6190,0.2238,0.2238,0.4174
3,A,E,209,115,55,56,217,73.6842,0.2632,0.2632,0.4870
4,A,F,208,115,47,50,217,77.4038,0.2260,0.2260,0.4348
...,...,...,...,...,...,...,...,...,...,...,...
115,M,T,212,115,46,47,217,78.3019,0.2170,0.2170,0.4087
116,M,U,210,115,44,49,217,79.0476,0.2095,0.2095,0.4261
117,N,T,211,115,54,56,217,74.4076,0.2559,0.2559,0.4870
118,N,U,209,115,52,58,217,75.1196,0.2488,0.2488,0.5043


In [108]:
'''
import numpy as np 


for index, row in df.iterrows():
    #print("index :", index)
    #print("row value : ", row)
    print("row A :" , row[0])
    print("ordinate : ",  row[0].lower())
    print("new orifnate : ", ord(row[0].lower())%32)
    print("row A :" , row[1])
    print("ordinate : ",  row[1].lower())
    print("new orifnate : ", ord(row[1].lower())%32)
    #print(type(row[0]))
    #print("row B : ", row[1])
    #print("fracVar : ", row[10])

'''

'\nimport numpy as np \n\n\nfor index, row in df.iterrows():\n    #print("index :", index)\n    #print("row value : ", row)\n    print("row A :" , row[0])\n    print("ordinate : ",  row[0].lower())\n    print("new orifnate : ", ord(row[0].lower())%32)\n    print("row A :" , row[1])\n    print("ordinate : ",  row[1].lower())\n    print("new orifnate : ", ord(row[1].lower())%32)\n    #print(type(row[0]))\n    #print("row B : ", row[1])\n    #print("fracVar : ", row[10])\n\n'

In [103]:
from scipy import sparse

def create_dict(df):  ####this could be so much more effient... screw it... going to go with it. 
    translator =  {}
    counter = 0
    for index, row in df.iterrows():
        if row[0] not in translator.keys():
            translator[row[0]]=counter
            counter+=1
        else:
            continue
    
    translator['U']=counter
    return translator
translator = create_dict(df)
print(len(translator))
        
def adjacency_matrix(df, translator):
       
        dicter = {}
        row = []
        col = []
        data = []

        for index, val in df.iterrows():
            row.append(translator[val[0]])  #best I have for now. I dont like doing it this way
            col.append(translator[val[1]])
            data.append(val[10])
            dicter[translator[val[0]],translator[val[1]]]=val[10]
            ## the data is actually nxn .. hence it is a square matrix nxn.. so this
            ## makes the adjacnecy matrix bidirectional
            row.append(translator[val[1]])
            col.append(translator[val[0]])
            data.append(val[10])
            dicter[translator[val[1]],translator[val[0]]]=val[10]



        shape = (len(row), len(col))
        A = sparse.coo_matrix((data, (row, col)), shape=shape)

        return A, row, col, data, dicter
    



adjac, row, col, data, dicter = adjacency_matrix(df, translator)

def create_numpy_matrix(translator, dicter):
    a_shape = (len(translator),len(translator))
    a= np.zeros(a_shape)
    for i,j in dicter:
        a[i,j]=dicter[i,j]
    return a
numpy_matrix_form = create_numpy_matrix(translator, dicter)
print(numpy_matrix_form)


16
[[0.     0.3739 0.3478 0.4174 0.487  0.4348 0.487  0.4522 0.3913 0.4348
  0.4    0.487  0.4087 0.5043 0.4696 0.3913]
 [0.3739 0.     0.3304 0.4435 0.5391 0.4522 0.5652 0.4957 0.4    0.4957
  0.4087 0.513  0.513  0.4609 0.5565 0.513 ]
 [0.3478 0.3304 0.     0.5043 0.4783 0.4261 0.5565 0.4174 0.2957 0.4
  0.3739 0.4609 0.4348 0.4609 0.4696 0.4348]
 [0.4174 0.4435 0.5043 0.     0.5217 0.4435 0.4696 0.4609 0.4    0.4609
  0.4435 0.487  0.4609 0.5217 0.4261 0.5043]
 [0.487  0.5391 0.4783 0.5217 0.     0.487  0.5391 0.4696 0.4957 0.4435
  0.4174 0.5826 0.4348 0.487  0.4609 0.4696]
 [0.4348 0.4522 0.4261 0.4435 0.487  0.     0.487  0.3565 0.4783 0.3826
  0.4261 0.4696 0.4174 0.4957 0.3739 0.4783]
 [0.487  0.5652 0.5565 0.4696 0.5391 0.487  0.     0.5217 0.5826 0.5304
  0.5304 0.5391 0.4609 0.4348 0.4609 0.5565]
 [0.4522 0.4957 0.4174 0.4609 0.4696 0.3565 0.5217 0.     0.4696 0.2609
  0.4609 0.3565 0.4783 0.5391 0.3913 0.4783]
 [0.3913 0.4    0.2957 0.4    0.4957 0.4783 0.5826 0.4696 0.    

In [107]:
import numpy as np



eigenvalues, eigenvectors = np.linalg.eig(numpy_matrix_form)


print(" The eigenvalues are : \n", eigenvalues)






 The eigenvalues are : 
 [ 6.95222044 -0.84929103 -0.83093672 -0.67670011 -0.21250884 -0.23379386
 -0.54617273 -0.51921053 -0.48542044 -0.29472874 -0.44906234 -0.42939452
 -0.39338383 -0.36960207 -0.3340797  -0.32793498]
